In [2]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=0c9da020baa66b92f7e72c186de4349e577141426a5543df68b7da600e9a14f3
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [3]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC

from bayes_opt import BayesianOptimization
from bayes_opt.util import Colours

In [4]:
def get_data():
    """Synthetic binary classification dataset."""
    data, targets = make_classification(
        n_samples=1000,
        n_features=45,
        n_informative=12,
        n_redundant=7,
        random_state=134985745,
    )
    return data, targets

In [5]:
def svc_cv(C, gamma, data, targets):
    """SVC cross validation.
    This function will instantiate a SVC classifier with parameters C and
    gamma. Combined with data and targets this will in turn be used to perform
    cross validation. The result of cross validation is returned.
    Our goal is to find combinations of C and gamma that maximizes the roc_auc
    metric.
    """
    estimator = SVC(C=C, gamma=gamma, random_state=2)
    cval = cross_val_score(estimator, data, targets, scoring='roc_auc', cv=4)
    return cval.mean()

In [6]:
def rfc_cv(n_estimators, min_samples_split, max_features, data, targets):
    """Random Forest cross validation.
    This function will instantiate a random forest classifier with parameters
    n_estimators, min_samples_split, and max_features. Combined with data and
    targets this will in turn be used to perform cross validation. The result
    of cross validation is returned.
    Our goal is to find combinations of n_estimators, min_samples_split, and
    max_features that minimzes the log loss.
    """
    estimator = RFC(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_features=max_features,
        random_state=2
    )
    cval = cross_val_score(estimator, data, targets,
                           scoring='neg_log_loss', cv=4)
    return cval.mean()


In [7]:
def optimize_svc(data, targets):
    """Apply Bayesian Optimization to SVC parameters."""
    def svc_crossval(expC, expGamma):
        """Wrapper of SVC cross validation.
        Notice how we transform between regular and log scale. While this
        is not technically necessary, it greatly improves the performance
        of the optimizer.
        """
        C = 10 ** expC
        gamma = 10 ** expGamma
        return svc_cv(C=C, gamma=gamma, data=data, targets=targets)

    optimizer = BayesianOptimization(
        f=svc_crossval,
        pbounds={"expC": (-3, 2), "expGamma": (-4, -1)},
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=10)

    print("Final result:", optimizer.max)

In [8]:
def optimize_rfc(data, targets):
    """Apply Bayesian Optimization to Random Forest parameters."""
    def rfc_crossval(n_estimators, min_samples_split, max_features):
        """Wrapper of RandomForest cross validation.
        Notice how we ensure n_estimators and min_samples_split are casted
        to integer before we pass them along. Moreover, to avoid max_features
        taking values outside the (0, 1) range, we also ensure it is capped
        accordingly.
        """
        return rfc_cv(
            n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=max(min(max_features, 0.999), 1e-3),
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=rfc_crossval,
        pbounds={
            "n_estimators": (10, 250),
            "min_samples_split": (2, 25),
            "max_features": (0.1, 0.999),
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=10)

    print("Final result:", optimizer.max)

In [9]:
# Generate synthetic binary classification dataset
data, targets = get_data()

In [11]:
print(Colours.yellow("--- Optimizing SVM ---"))
optimize_svc(data, targets)

--- Optimizing SVM ---
|   iter    |  target   |   expC    | expGamma  |
-------------------------------------------------
|  1        |  0.882    | -2.042    | -2.134    |
|  2        |  0.9142   | -0.8114   | -1.644    |
|  3        |  0.9402   |  0.8999   | -3.182    |
|  4        |  0.9174   | -1.618    | -1.594    |
|  5        |  0.9345   |  1.791    | -1.372    |
|  6        |  0.9007   |  2.0      | -4.0      |
|  7        |  0.9091   | -3.0      | -1.0      |
|  8        |  0.8354   | -0.4191   | -4.0      |
|  9        |  0.9716   |  2.0      | -2.542    |
|  10       |  0.9729   |  1.468    | -2.346    |
|  11       |  0.9711   |  1.667    | -2.649    |
|  12       |  0.973    |  1.827    | -2.277    |
|  13       |  0.9728   |  1.69     | -2.364    |
|  14       |  0.9725   |  1.692    | -2.404    |
|  15       |  0.9729   |  1.947    | -2.344    |
Final result: {'target': 0.9729595360983103, 'params': {'expC': 1.8272363367085624, 'expGamma': -2.276983983143419}}


In [12]:
print(Colours.green("--- Optimizing Random Forest ---"))
optimize_rfc(data, targets)

--- Optimizing Random Forest ---
|   iter    |  target   | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        | -0.3777   |  0.2722   |  16.31    |  115.1    |
|  2        | -0.3648   |  0.806    |  19.94    |  75.42    |
|  3        | -0.3774   |  0.3485   |  20.44    |  240.0    |
|  4        | -0.3557   |  0.8875   |  10.23    |  130.2    |
|  5        | -0.3664   |  0.7144   |  18.39    |  98.86    |
|  6        | -0.6459   |  0.999    |  2.0      |  10.0     |
|  7        | -0.3528   |  0.5371   |  8.287    |  127.9    |
|  8        | -0.3504   |  0.999    |  2.0      |  249.6    |
|  9        | -0.3541   |  0.5048   |  2.02     |  198.8    |
|  10       | -0.3543   |  0.8738   |  2.05     |  82.07    |
|  11       | -0.3547   |  0.9692   |  2.023    |  159.8    |
|  12       | -0.3514   |  0.9986   |  2.337    |  225.9    |
|  13       | -0.3501   |  0.8352   |  2.082    |  114.9    |
|  14       | -0.3727   |  0.9774   |